# ExplorerLand data
https://explorer.land

In [ ]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import shape, Polygon, MultiPolygon, LineString, MultiLineString, Point, MultiPoint
import ast

In [ ]:
df = pd.read_csv("../input/ExplorerLand/explorer_land.csv")
df.head(1)

In [ ]:
df.info()

### Fix geometries

In [ ]:
df['geometry_reported'] = df['geometry_reported'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

In [ ]:
df['geometry'] = df['geometry_reported'].apply(shape)

In [ ]:
gdf = gpd.GeoDataFrame(df, geometry='geometry', crs = 'EPSG:4326')
gdf = gdf.drop(columns='geometry_reported')

In [ ]:
gdf = gdf.explode(index_parts=False).explode(index_parts=False).reset_index(drop = True)

In [ ]:
gdf['geometry'] = gdf['geometry'].make_valid()

In [ ]:
gdf['geometry'].type.unique()

Turn linestrings to polygons

In [ ]:
gdf['geometry'] = gdf['geometry'].apply(
    lambda geom: Polygon(list(geom.coords) + [geom.coords[0]]) if isinstance(geom, LineString) and not geom.is_closed and len(geom.coords) > 0 else
                 Polygon(geom.coords) if isinstance(geom, LineString) and geom.is_closed else
                 geom
)

In [ ]:
gdf.plot()

### Harmonize nomenclature

In [ ]:
gdf = gdf.rename(columns = {'description_reported':'project_description_reported'})
gdf['host_name'] = 'Explorer Land'
gdf['url'] = 'https://explorer.land/x/projects'
gdf['site_sqkm'] = gdf['geometry'].to_crs('EPSG:3857').area / 1e6
gdf['site_sqkm'].describe()

### Save it

In [ ]:
gdf.to_file("../midsave/explorer_land.gpkg")